<a href="https://colab.research.google.com/github/ssnirgudkar/Home-AIML/blob/main/Readimagesegmentio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade segments-ai

In [ ]:
from segments import SegmentsClient
api_key = "a89182567b17766b91773021b18d04574cd75109"
client = SegmentsClient(api_key)

In [3]:
# gives all datasets that are on segment io for this user
user = "ssnirgudkar"
datasets = client.get_datasets(user)
for dataset in datasets:
    print(dataset["name"],dataset["description"])

playground This dataset contains some images for trying out the Segments.ai labeling tool.
Test 
PilotIR Data set of 103 IR images for segmentation purpose
ReferenceColorImagesForIRImages 
IRGray-100-set2-3classes IR images, Gray scale, batch of 100, 3 classes only
tomatoes test on tomatoes


In [5]:
dataset_identifier = "ssnirgudkar/IRGray-100-set2-3classes"
dataset = client.get_dataset(dataset_identifier)

In [9]:
from segments import SegmentsDataset
from segments.utils import export_dataset

# Initialize a SegmentsDataset from the release file
release_file = client.get_release(dataset_identifier,'Model_Labeled_Images')
#print(release_file)
releasedataset = SegmentsDataset(release_file, labelset='ground-truth', filter_by='labeled')

  0%|          | 0/4 [00:00<?, ?it/s]

Initializing dataset. This may take a few seconds...


100%|██████████| 4/4 [00:00<00:00,  7.83it/s]

Initialized dataset with 4 images.


In [10]:
# prints the image name and the labels present on the image 
import matplotlib.pyplot as plt 
from segments.utils import get_semantic_bitmap
for sample in releasedataset:
  print(sample['name'])
  print (sample['annotations'])

1603391862.128166_color.png
[{'id': 1, 'category_id': 2}, {'id': 2, 'category_id': 0}, {'id': 3, 'category_id': 0}, {'id': 4, 'category_id': 0}, {'id': 5, 'category_id': 1}, {'id': 6, 'category_id': 2}]
tomato_PNG12549.png
[{'id': 1, 'category_id': 0}, {'id': 2, 'category_id': 0}, {'id': 3, 'category_id': 0}]
1603391920.193870_color.png
[{'id': 1, 'category_id': 0}, {'id': 2, 'category_id': 0}, {'id': 3, 'category_id': 1}, {'id': 4, 'category_id': 1}, {'id': 5, 'category_id': 0}, {'id': 6, 'category_id': 2}, {'id': 7, 'category_id': 0}]
1603391973.126007_color.png
[{'id': 1, 'category_id': 0}, {'id': 2, 'category_id': 0}, {'id': 3, 'category_id': 1}, {'id': 4, 'category_id': 2}, {'id': 5, 'category_id': 0}, {'id': 6, 'category_id': 0}, {'id': 7, 'category_id': 0}, {'id': 8, 'category_id': 0}, {'id': 9, 'category_id': 0}, {'id': 10, 'category_id': 0}]


In [ ]:
# this is to print the pixel values of the labeled images for checking if the category ids are adhered to
import cv2 
 import numpy as np
 import os 
 
 for filesindir in os.listdir("/content/segments/ssnirgudkar_PilotIR/V2.0/"):
   if filesindir.endswith("_label_segmentation.png"):
      
      filetext = filesindir.rstrip('_label_segmentation.png')
      filename = filetext+'.png'
      #print(filename)
      #filename = '1602782427.667751.png'
      for sample in releasedataset:
        if sample['name'] == filename:
          #print('in the loop')
          #print(sample['name'])
          #get filename for reading inout file and writing outputfile. remove .png from the filename 
          str = sample['name'].rstrip('.png')
          ##print(str) 
          labeledfile = str+'_label_segmentation.png'
          
     #t1=sample['name']
     #print(type(sample))  -- this is a dictionary 
     #print(type(t1)) -- this is a string 
     #print(t1[1])  -- this prints the 2nd digit in the image name 
     #print(sample['annotations'])  - this prints the values under annotations key for that image 
     #print(type(sample['annotations'])) -- this is a list 
     #print(l1[0])  -- this prints all the attributes for the 1st key 
     #print(type(l1[0])) -- this is dictionary 
     #print(l1[1]['id'],l1[1]['category_id']) - prints the id and category id for the 2nd key 
     #open the image, read each pixel, for all pixels in the image, run a for loop 
     #over and over on this 1 annotation to convert the pixel value 
          l1=sample['annotations']
          #print(l1)
     #print (l1[1]['id'], l1[1]['category_id'])
          
     #finding max of instance ids for an image 
          #print(len(l1)) 
          #id
          maxinstanceidinimage = len(l1)
          infilename = "/content/segments/ssnirgudkar_PilotIR/V2.0/"+labeledfile
          img = cv2.imread(infilename, cv2.IMREAD_UNCHANGED)
          assert not isinstance(img,type(None)), 'image not found'
          print ("file read = {0}".format(infilename))
          
          img2 = img.copy()
          #print(img[162,:])
          #print(img)
     #print(img.dtype)
          height,width = img.shape
     #print (img.shape)
     
          for i in range(height):
            for j in range(width):
              if img.item(i,j) == 0:
                img2.itemset((i,j), 0)
     
              else:
                  for m in range (0,maxinstanceidinimage,1):
                    #print (l1[m]['id'])
                    #if pixel value is found in category, replace with category 
                    if img.item(i,j) == l1[m]['id']:
                       img2.itemset((i,j), l1[m]['category_id']) 
                       #now collapse 6 categories into 3 - 7 - ob, 8 - sky, 9 - water

                       if img2.item(i,j) == 0:
                          img2.itemset((i,j), 7)  #object -> object 
                       if img2.item(i,j) == 1:
                          img2.itemset((i,j), 8)  #sky -> sky 
                       if img2.item(i,j) == 2:
                          img2.itemset((i,j), 9)  #water -> water
                       if img2.item(i,j) == 3:
                          img2.itemset((i,j), 7)  #warm entity -> object
                       if img2.item(i,j) == 4:
                          img2.itemset((i,j), 7)  #backgnd -> object
                       if img2.item(i,j) == 5:
                          img2.itemset((i,j), 7)  #cold entity -> object
                       if img2.item(i,j) == 6:
                          img2.itemset((i,j), 8)  #sun -> sky
                          
              
                         
                          
          #img.itemset((i,j), arr1[img.item(i,j)-1])  
          #print('out of the translation loop')   
       
          outputfilename = '/content/drive/MyDrive/image-masked-output/'+str+'_relabeled.png'
          cv2.imwrite(outputfilename, img2)  
          print ("file written = {0}".format(outputfilename))
          #print(img[5,:])
          #print(img2[162,:])
          #print(img)
          #print('new file written')